In [1]:
### import necessary libraries
import ee
import json
import geemap
import sklearn
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

ee.Authenticate()
ee.Initialize(project="ee-franciscofurey")

In [2]:
# Load Sentinel-2 image collection
s2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
gcps = ee.FeatureCollection('projects/ee-franciscofurey/assets/cordoba-urban-water-vegetation')
# Define the geometry
geometry = ee.Geometry.Polygon([[
        [-64.5153195400129,-31.29532373904396],
        [-64.5153195400129,-31.476544788684357],
        [-64.09462534855581,-31.476544788684357],
        [-64.09462534855581,-31.29532373904396],
        [-64.5153195400129,-31.29532373904396]
    ]])

# Define visualization parameters
rgbVis = {
  'min': 0.0,
  'max': 3000,
  'bands': ['B4', 'B3', 'B2']
}

# Filter the image collection
filtered = s2 \
  .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 30)) \
  .filter(ee.Filter.date('2019-01-01', '2020-01-01')) \
  .filter(ee.Filter.bounds(geometry)) \
  .select('B.*')

# Calculate the median composite
composite = filtered.median().clip(geometry)

In [3]:
# Overlay the points on the image to get training data.
training = composite.sampleRegions(
    collection=gcps,
    properties=['land_cover'],
    scale=10,
    tileScale= 16
)

# Train the classifier.
classifier = ee.Classifier.smileRandomForest(numberOfTrees=50).train(
    features=training,
    classProperty='land_cover',
    inputProperties=composite.bandNames()
)

# Classify the image.
classified = composite.classify(classifier)

# Select the Water class.
water = classified.eq(2)

palette = ['#ff0000', '#0000ff', '#008000']
vis_params = {'min': 0,'max': 2,'palette': palette}
water_vis = {'min': 0, 'max': 1, 'palette': ['white', 'blue']}

# Display the input composite
Map = geemap.Map()
Map.centerObject(geometry)
# And this would display the water class.
Map.addLayer(classified.clip(geometry), vis_params, '2019')
# Map.addLayer(water.clip(geometry), water_vis, 'Water')
Map


Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

<IPython.core.display.Javascript object>